<a href="https://colab.research.google.com/github/jal9o3/gpt2-rag/blob/main/Webpage_RAG_Query_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 RAG Experiments
This notebook walks through my experimentation with utilizing a RAG approach with the open source GPT-2 model.

# Scraping Code
This is where we use requests and BeautifulSoup to retrieve the page content.

We also parse sentences using NLTK.

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/Retrieval-augmented_generation')
r.text

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup.prettify())

In [ ]:
raw_paragraphs = soup.find_all("p")

In [ ]:
print(raw_paragraphs)

In [ ]:
paragraphs = ""
for raw_paragraph in raw_paragraphs:
    paragraphs += "\n" + raw_paragraph.text
print(paragraphs)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
sentences = sent_tokenize(paragraphs)
print(sentences)

# Embedding Code
Here we generate embeddings using a sentence transformer model from Hugging Face.

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

In [ ]:
len(embeddings)

# Pinecone VectorDB Code
Here we upsert the embeddings to PineconeDB and run a similarity search.

In [ ]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PC_API_KEY')

In [ ]:
!pip install "pinecone-client[grpc]"

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# pc.delete_index("webpage-index")

In [ ]:
index_name = "webpage-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimensions of embeddings
        metric="cosine", # used by the embedding model
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [ ]:
index = pc.Index(index_name)

vectors = []
for i, embedding in enumerate(embeddings):
  vectors.append({"id": f"vec{i}", "values": embedding})

index.upsert(
    vectors,
    namespace="webpage-1"
)

In [ ]:
print(index.describe_index_stats())

In [ ]:
index_stats = index.describe_index_stats()
# Account for upsert delay by checking the number of vectors in DB
len(embeddings) == index_stats['namespaces']['webpage-1']['vector_count']

In [ ]:
# Generate an embedding of the prompt
prompt = "What is Retrieval Augmented Generation?"
vector = model.encode([prompt])[0]

In [ ]:
print(vector)

In [ ]:
query_results = index.query(
    namespace="webpage-1",
    vector=vector,
    top_k=2,
    include_values=True
)

#print(query_results)

for query_match in query_results['matches']:
  print(query_match['id'])

In [ ]:
# Parse result indexes using regex (temporary ;))
import re
id_nums = []
for query_match in query_results['matches']:
  id_nums.append(int(re.findall(r'\d+', query_match['id'])[0]))
print(id_nums)

In [ ]:
# Construct background info for GPT-2
background = sentences[0]
for id in id_nums:
  if id > 0:
    background += " " + sentences[id]
print(background)

# GPT-2 Code
This is where we use the GPT-2 model from Hugging Face.

In [ ]:
!pip install torch

In [ ]:
extended_prompt = f"""
{background.strip()} {prompt}
"""
print(extended_prompt)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

#model = AutoModelForCausalLM.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", torch_dtype=torch.float16, attn_implementation="sdpa")
tokenizer = AutoTokenizer.from_pretrained("gpt2")


input_ids = tokenizer(extended_prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

In [ ]:
print(gen_text)